In [30]:
import tensorflow as tf
import numpy as np
import random
from konlpy.tag import Mecab; m = Mecab()
import sys
import os

In [31]:
tf.__version__

'1.0.0'

# 참고 

김진중님 코드 : https://github.com/golbin/TensorFlow-Tutorials/blob/master/06%20-%20RNN%2C%20ChatBot/03%20-%20Seq2Seq.py

In [2]:
ONE_HOT = True

# 데이터 로드 및 전처리 

batch = [batch_size,time_step, input_dimension] 으로 맞춰준다..! <br>
후에 다시 인코더에 넣을 때는, [time_step,batch_size,input_dimension]으로 바꿔줘야 함

In [32]:
with open('./data/dialogue_sample.txt', 'r', encoding='utf-8') as f:
    raw = f.readlines()

conv=[]
for r in raw:
    if r !='\n':
        conv.append(r)

raw_X = [r.split('\\t')[0] for r in conv] # 리얼 인풋
raw_y = [r.split('\\t')[1][:-1] for r in conv] # 리얼 아웃풋

In [33]:
def one_hot(x,Vocab_size):
    return np.identity(Vocab_size)[x:x+1]

def vocab_encode(text, vocab):
    """
    text가 Vocab(characters) 안에 있으면 index로 변환
    """
    return [vocab.index(x) for x in text if x in vocab]

def vocab_decode(array, vocab):
    """
    index를 다시 문자열로 디코딩
    """
    return ''.join([vocab[x - 1] for x in array])

def posTag(sentences):
    result = []
    for sent in sentences:
        temp = m.morphs(sent)
        result.append(temp)
    return result

flatten = lambda l: [item for sublist in l for item in sublist]

In [35]:
raw_X[4]

'오늘 스케줄이 있는가?'

In [36]:
X = posTag(raw_X)
y = posTag(raw_y)
tokenset = flatten(X+y)
tokenset.extend(['<UNK>','<NUM>','<PAD>','<END>'])
tokenset = list(set(tokenset))

In [37]:
X[4]

['오늘', '스케줄', '이', '있', '는가', '?']

In [38]:
len(tokenset)

268

In [39]:
for x in X:
    left = 10-len(x)
    x.extend(['<PAD>']*left)
    
for y_ in y:
    y_.append('<END>')
    left = 10-len(y_)
    y_.extend(['<PAD>']*left)

In [40]:
X[4]

['오늘', '스케줄', '이', '있', '는가', '?', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

In [41]:
x_batch = [vocab_encode(x,tokenset) for x in X]
y_batch = [vocab_encode(y_,tokenset) for y_ in y]
target_batch = [vocab_encode(y_,tokenset) for y_ in y]

In [42]:
x_batch[4]

[196, 224, 90, 15, 122, 232, 207, 207, 207, 207]

In [43]:
vocab_size = len(tokenset)
batch_size = len(x_batch)

In [44]:
if ONE_HOT:
    for x in range(len(x_batch)):
        for j in range(len(x_batch[x])):
            x_batch[x][j] = one_hot(x_batch[x][j],vocab_size)

    for y_ in range(len(y_batch)):
        for j in range(len(y_batch[y_])):
            y_batch[y_][j] = one_hot(y_batch[y_][j],vocab_size)

In [47]:
x_batch = [np.array(x) for x in x_batch]
y_batch = [np.array(y_) for y_ in y_batch]

x_batch = np.squeeze(x_batch)
y_batch = np.squeeze(y_batch)

In [48]:
print(x_batch.shape)
print(y_batch.shape)

(100, 10, 268)
(100, 10, 268)


# 모델 빌드 

batch = [time_step,batch_size,input_dimension]

In [49]:
n_classes = vocab_size # one-hot이니까..
#n_input = 50 # embedding
n_hidden = 128
n_layers = 3
time_step = 10
n_step = 2000

# Placeholder, Variable 

In [15]:
#########
# 신경망 모델 구성
######
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# [batch size, time steps, input size]

#enc_input = tf.placeholder(tf.float32, [None, None, n_input])
#dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]


if ONE_HOT:
    enc_input = tf.placeholder(tf.float32, [None, None, vocab_size])
    dec_input = tf.placeholder(tf.float32, [None, None, vocab_size])
    targets = tf.placeholder(tf.int64, [None, None])
    
    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
    
    W = tf.Variable(tf.ones([n_hidden, n_classes]))
    b = tf.Variable(tf.zeros([n_classes]))

    # tf.nn.dynamic_rnn 옵션에서 time_major 값을 True 로 설정
    # [batch_size, n_steps, n_input] -> Tensor[n_steps, batch_size, n_input]
    enc_input = tf.transpose(enc_input, [1, 0, 2])
    dec_input = tf.transpose(dec_input, [1, 0, 2])
    
else:
    x_input = tf.placeholder(tf.int32, [None, time_step])
    y_input = tf.placeholder(tf.int32, [None, time_step])
    targets = tf.placeholder(tf.int64, [None, None])
    
    embed_matrix = tf.Variable(tf.random_uniform([vocab_size, n_input], -1.0, 1.0),name='embed_matrix')

    enc_input = tf.nn.embedding_lookup(embed_matrix, x_input, name='embed')                                                                    
    dec_input = tf.nn.embedding_lookup(embed_matrix, y_input, name='embed')  

    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    W = tf.Variable(tf.ones([n_hidden, n_classes]))
    b = tf.Variable(tf.zeros([n_classes]))

    # tf.nn.dynamic_rnn 옵션에서 time_major 값을 True 로 설정
    # [batch_size, n_steps, n_input] -> Tensor[n_steps, batch_size, n_input]
    enc_input = tf.transpose(enc_input, [1, 0, 2])
    dec_input = tf.transpose(dec_input, [1, 0, 2])

In [16]:
with tf.variable_scope('encode'):
    enc_cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    enc_cell = tf.contrib.rnn.MultiRNNCell([enc_cell] * n_layers)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input ,
                                            dtype=tf.float32)

# 디코더 셀을 구성한다.
with tf.variable_scope('decode'):
    dec_cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    dec_cell = tf.contrib.rnn.MultiRNNCell([dec_cell] * n_layers)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)

# Cost, Op_f 

[batch size, time steps, hidden layers] <br>
[batch size\*time_steps, hidden layer] <br>
[batch size * time steps, class numbers] <br>
[batch size, time steps, class numbers]

In [17]:
# sparse_softmax_cross_entropy_with_logits 함수를 사용하기 위해
# 각각의 텐서의 차원들을 다음과 같이 변형하여 계산한다.
#    -> [batch size, time steps, hidden layers]
time_steps = tf.shape(outputs)[1]
#    -> [batch size * time steps, hidden layers]
outputs_trans = tf.reshape(outputs, [-1, n_hidden])
#    -> [batch size * time steps, class numbers]
logits = tf.matmul(outputs_trans, W) + b
#    -> [batch size, time steps, class numbers]
logits = tf.reshape(logits, [-1, time_steps, n_classes])

In [18]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels= targets))
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost,global_step=global_step)

# Training 

In [17]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

ckpt = tf.train.get_checkpoint_state(os.path.dirname('ckp/05/checkpoint'))
if ckpt and ckpt.model_checkpoint_path:
    print('restore')
    saver.restore(sess, ckpt.model_checkpoint_path)

for epoch in range(n_step):
    _, loss = sess.run([train_op, cost],
                        feed_dict={enc_input: x_batch, #one-hot enc_input:
                                   dec_input: y_batch, #one-hot dec_input:
                                   targets: target_batch})
    sys.stdout.write("\rEpoch: {} , cost = {}".format(epoch+1,loss))
    sys.stdout.flush()
global_step_for_save = sess.run(global_step)
saver.save(sess, 'ckp/05/simple_seq.model', global_step_for_save)
print('\n최적화 완료!')

Epoch: 2000 , cost = 0.6143342852592468
최적화 완료!


# Test 

In [20]:
sess = tf.Session()
ckpt = tf.train.get_checkpoint_state(os.path.dirname('ckp/05/checkpoint'))
if ckpt and ckpt.model_checkpoint_path:
    print('Ready!')
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print('Please save session')

Ready!


In [50]:
def prediction_test(index):
    prediction = tf.argmax(logits, 2)
    prediction_check = tf.equal(prediction, targets)
    accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

    real, predict, accuracy_val = sess.run([targets, prediction, accuracy],
                                           feed_dict={enc_input: x_batch[index:index+1], #one-hot enc_input:
                                                      dec_input: y_batch[index:index+1], #one-hot dec_input:
                                                      targets: target_batch[index:index+1]})
    
    
    print_pred = []
    for p in predict[0]:
        dec = tokenset[p]
        if dec == '<END>':
            break
        print_pred.append(dec)
    
    print("\n=== 예측 결과 ===")
    # print('입력값:', X[index])
    #print('실제값:', [[tokenset[j] for j in dec] for dec in real])
    #print('예측값:', [[tokenset[i] for i in dec] for dec in predict])
    #print('정확도:', accuracy_val)
    
    print('입력값:',raw_X[index])
    print('실제값:',raw_y[index])
    
    
    print('예측값:',' '.join(print_pred))

In [52]:
randomNum = random.choice(range(batch_size))
prediction_test(randomNum)


=== 예측 결과 ===
입력값: 오호 괜찮네
실제값: 감사합니다
예측값: 얘 에서 끝내 야 야 고마워 뭔데 고마워 뭔데 에서


# TODO 

패딩으로 인풋과 아웃풋의 길이를 고정했음.!! loss에 padding도 포함되기 때문에 정확한 loss가 아니다?!  <br>
TF에서 제공하는 PAD도 사용해보기 <br>
또한 인풋의 차원을 이랬다가저랬다가 너무나 복잡한것.... 좀 더 쉬운 방법 없나? <br>
reshape와 transpose를 확실하게 정리하자
Tensorarray ?<br>
인풋을 임베딩 매트릭스로 lookup 하고 싶은데..? (이것도 jointly training)

아니 왜 placeholder에 다른 차원을 흘려보낸다음에 걔를 reshape하면 수렴을 안하는거지?